Tải bộ dữ liệu về

In [1]:
!wget --no-check-certificate \
    https://gist.githubusercontent.com/khacanh/4c4662fa226db87a4664dfc2f70bc63e/raw/5d8a1d890c73a1e92e6898137db28f3dc0676975/kieu.txt \
    -O ./kieu.txt

--2025-06-02 09:13:31--  https://gist.githubusercontent.com/khacanh/4c4662fa226db87a4664dfc2f70bc63e/raw/5d8a1d890c73a1e92e6898137db28f3dc0676975/kieu.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135832 (133K) [text/plain]
Saving to: ‘./kieu.txt’

./kieu.txt          100%[===================>] 132.65K  --.-KB/s    in 0.02s   

2025-06-02 09:13:31 (5.89 MB/s) - ‘./kieu.txt’ saved [135832/135832]



Gán 1 biến cho file txt mới tạo


In [ ]:
corpus = []
f = open("kieu.txt", "r")
for line in f:
  corpus.append(line)

In [3]:
import string as pystring
from nltk import everygrams

Bỏ dấu thừa và khoảng trắng thừa

In [4]:
PUNCT_TO_REMOVE = pystring.punctuation + pystring.digits + "\n"
def clean_text(text):
    """custom function to removal: punctuations and digits"""
    text = text.translate(str.maketrans(' ', ' ', PUNCT_TO_REMOVE))
    text = text.lower()
    return text
clean_text(corpus[0])

'trăm năm trong cõi người ta'

Chữ kèm với label sau nó và mỗi câu có 1 label tương ứng

In [5]:
def get_contexts_and_labels(corpus):
    contexts, labels = [], []
    for line in corpus:
        line = clean_text(line)
        line = line.split()
        if len(line) < 2:
            continue
        ngrams = list(everygrams(line, min_len=2, max_len=len(line)))
        for gram in ngrams:
            context = " ".join(list(gram[:-1]))
            label = gram[-1]
            contexts.append(context)
            labels.append(label)
    return contexts, labels

In [6]:
contexts, labels = get_contexts_and_labels(corpus)
contexts[:5], labels[:5]

(['trăm',
  'trăm năm',
  'trăm năm trong',
  'trăm năm trong cõi',
  'trăm năm trong cõi người'],
 ['năm', 'trong', 'cõi', 'người', 'ta'])

In [7]:
len(contexts), len(labels)

(69961, 69961)

In [10]:
from sklearn.model_selection import train_test_split

train_sentences, valid_sentences, train_labels, valid_labels = train_test_split(contexts, labels, test_size=0.2, random_state=42)

In [11]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [12]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(train_sentences)

In [15]:
vocab_size = len(tokenizer.index_word) + 1
embedding_dim = 200
max_length = max([len(context.split()) for context in contexts])
max_length

7

In [16]:
train_seq = tokenizer.texts_to_sequences(train_sentences)
train_seq_pad = pad_sequences(train_seq, maxlen=max_length, truncating='post', padding="post")

valid_seq = tokenizer.texts_to_sequences(valid_sentences)
valid_seq_pad = pad_sequences(valid_seq, maxlen=max_length, truncating='post', padding="post")

In [18]:
def convert_labels(labels):
    convert_labels = []
    for label in labels:
        if label in tokenizer.index_word.values():
            convert_labels.append(to_categorical( tokenizer.word_index[label], len(tokenizer.word_index)+1))
        else:
            convert_labels.append(to_categorical( tokenizer.word_index["<OOV>"], len(tokenizer.word_index)+1))
    return convert_labels

In [19]:
train_labels = convert_labels(train_labels)
valid_labels = convert_labels(valid_labels)

In [20]:
train_labels = np.array(train_labels)
valid_labels = np.array(valid_labels)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

In [22]:
model_nplm = Sequential()
model_nplm.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model_nplm.add(Flatten())
model_nplm.add(Dense(10, activation='relu'))
model_nplm.add(Dense(vocab_size, activation='softmax'))
model_nplm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_nplm.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
model_nplm.fit(train_seq_pad, train_labels, epochs=30, validation_data=(valid_seq_pad, valid_labels))

Epoch 1/30
1749/1749 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.0115 - loss: 7.0146 - val_accuracy: 0.0169 - val_loss: 6.6972
Epoch 2/30
1749/1749 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.0170 - loss: 6.6078 - val_accuracy: 0.0207 - val_loss: 6.5787
Epoch 3/30
1749/1749 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0237 - loss: 6.4043 - val_accuracy: 0.0274 - val_loss: 6.4126
Epoch 4/30
1749/1749 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.0395 - loss: 6.1414 - val_accuracy: 0.0388 - val_loss: 6.2443
Epoch 5/30
1749/1749 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0576 - loss: 5.8397 - val_accuracy: 0.0517 - val_loss: 6.0963
Epoch 6/30
1749/1749 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0764 - loss: 5.5987 - val_accuracy: 0.0607 - val_loss: 6.0290
Epoch 7/30
1749/1749 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.0915 - loss: 5.3823 - val_accuracy: 0.0682 - val_loss: 5.9268
Epoch 8/30
1749/1749 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.1060 - loss: 5.2110 

In [24]:
model_nplm.save('model_nplm.h5')